In [1]:
import logging
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import torch.optim as optim
import transformers
from icecream import ic
from pandas import DataFrame
from peft import AutoPeftModelForCausalLM, PeftConfig, PeftModel
from torch import nn
from torch.utils import data
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

sys.path.append(os.path.abspath("/home/qcdong/codes/Qwen"))
from qwen_generation_utils import decode_tokens, make_context

ic.configureOutput(includeContext=True, argToStringFunction=str)
ic.lineWrapWidth = 120

logger = logging.getLogger()
logging.basicConfig(
    level=logging.INFO,
    datefmt="%y-%m-%d %H:%M",
    format="%(asctime)s %(filename)s %(lineno)d: %(message)s",
)


model_max_length = 768
model_name_or_path = "/mnt/nas1/models/qwen/Qwen-7B-Chat-Int8"
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_name_or_path,
    model_max_length=768,
    padding_side="right",
    use_fast=False,
    trust_remote_code=True,
)
tokenizer.pad_token_id = tokenizer.eod_id
ic("loads tokenizer done")
generation_config = GenerationConfig.from_pretrained(
    model_name_or_path, trust_remote_code=True
)
ic(generation_config)

peft_model_path = "/mnt/nas1/models/qwen/Qwen-7B-Chat-int8-moss-small"
config = PeftConfig.from_pretrained(peft_model_path)
ic(config)

load_basic_model = 0
load_in_normal_order = 0
if load_basic_model:
    model = AutoModelForCausalLM.from_pretrained(
        model_name_or_path, trust_remote_code=True
    )
    ic("loads base model done")
else:
    if load_in_normal_order:
        model = transformers.AutoModelForCausalLM.from_pretrained(
            model_name_or_path, trust_remote_code=True
        )
        ic("loads base model done")
        model = PeftModel.from_pretrained(model, peft_model_path)
        ic("loads peft model done")
    else:
        model = AutoPeftModelForCausalLM.from_pretrained(
            peft_model_path, trust_remote_code=True
        )
        ic("directly loads peft model done")
ic(model.device)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
ic(device)
model = model.to(device)
model.eval()
model.generation_config = generation_config

query = "重点领域产业关键与共性技术攻关工程有哪些?"
response, history = model.chat(tokenizer, query, history=None)
# base model response: 《“十四五”国家科技创新规划》提出，实施重点领域产业关键与共性技术攻关工程。聚焦芯片与集成电路、前沿新材料、生命健康、人工智能、新能源、节能环保等重点产业领域，强化前瞻性基础研究和引领性原创成果重大突破，实现重大标志性原创成果重大突破。
print(response)

# all_raw_text = ["重点领域产业关键与共性技术攻关工程有哪些"]
# batch_raw_text = []
# for q in all_raw_text:
#     raw_text, _ = make_context(
#         tokenizer,
#         q,
#         system="You are a helpful assistant.",
#         max_window_size=model.generation_config.max_window_size,
#         chat_format=model.generation_config.chat_format,
#     )
#     batch_raw_text.append(raw_text)

# batch_input_ids = tokenizer(batch_raw_text, padding="longest")
# batch_input_ids = torch.LongTensor(batch_input_ids["input_ids"]).to(model.device)
# ic(batch_input_ids.size())
# batch_out_ids = model.generate(
#     batch_input_ids,
#     return_dict_in_generate=False,
#     generation_config=model.generation_config,
# )
# padding_lens = [
#     batch_input_ids[i].eq(tokenizer.pad_token_id).sum().item()
#     for i in range(batch_input_ids.size(0))
# ]
# ic(padding_lens)
# batch_response = [
#     decode_tokens(
#         batch_out_ids[i][padding_lens[i] :],
#         tokenizer,
#         raw_text_len=len(batch_raw_text[i]),
#         context_length=(batch_input_ids[i].size(0) - padding_lens[i]),
#         chat_format="chatml",
#         verbose=False,
#         errors="replace",
#     )
#     for i in range(len(all_raw_text))
# ]
# print(batch_response)


/home/qcdong/anaconda3/envs/ll/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
ic| 99688295.py:44 in <module>- loads tokenizer done
ic| 99688295.py:48 in <module>
    generation_config: GenerationConfig {
                         "chat_format": "chatml",
                         "do_sample": true,
                         "eos_token_id": 151643,
                         "max_new_tokens": 512,
                         "max_window_size": 6144,
                         "pad_token_id": 151643,
                         "repetition_penalty": 1.1,
                         "top_k": 0,
                         "top_p": 0.8,
                         "trust_remote_code": true
                       }
ic| 99688295.py:52 in <module>
    config: LoraConfig(peft_type='LORA', auto_mapping=None, base_model_name_or_path=

中国在十四五规划中提出了“五大基础研究和关键技术突破”重点任务，其中涉及的重点领域产业关键与共性技术攻关工程包括：

1. 基础研究：人工智能、量子信息科学、生物技术、先进制造等前沿领域。

2. 关键核心技术：新一代信息技术、高端装备、新材料、新能源汽车、节能环保等战略性新兴产业的关键核心技术。

3. 共性技术：数字经济、生命健康、新一代信息基础设施等领域的重要共性技术。

4. 产业融合：将新兴技术与传统行业深度融合，促进传统产业转型升级，如智能制造、数字农业、智能医疗等。

5. 深度应用：在各领域深度应用新技术，推动经济高质量发展，如智慧城市、智慧交通、智能家居等。


In [2]:
from qwen_generation_utils import decode_tokens, make_context, get_stop_words_ids
from icecream import ic
ic.configureOutput(includeContext=True, argToStringFunction=str)
ic.lineWrapWidth = 120

stop_words_ids = []
max_window_size = generation_config.max_window_size
system = 'You are a helpful assistant.'
history = []
raw_text, context_tokens = make_context(
    tokenizer,
    query,
    history=history,
    system=system,
    max_window_size=max_window_size,
    chat_format=generation_config.chat_format,
)

stop_words_ids.extend(get_stop_words_ids(
    generation_config.chat_format, tokenizer
))

input_ids = torch.tensor([context_tokens]).to(model.device)
print(type(model))
outputs = model.generate(
            input_ids=input_ids,
            return_dict_in_generate=False,
            generation_config=generation_config,
        )
ic(len(outputs[0]))
ic(tokenizer.decode(outputs[0]))
response = decode_tokens(
    outputs[0],
    tokenizer,
    raw_text_len=len(raw_text),
    context_length=len(context_tokens),
    chat_format=generation_config.chat_format,
    verbose=False,
    errors='replace'
)

# as history is a copy of the user inputs,
# we can always return the new turn to the user.
# separating input history and output history also enables the user
# to implement more complex history management
history.append((query, response))
print(response)

<class 'peft.peft_model.PeftModelForCausalLM'>


ic| 2084782153.py:30 in <module>- len(outputs[0]): 542
ic| 2084782153.py:31 in <module>
    tokenizer.decode(outputs[0]): <|im_start|>system
                                  You are a helpful assistant.<|im_end|>
                                  <|im_start|>user
                                  重点领域产业关键与共性技术攻关工程有哪些?<|im_end|>
                                  <|im_start|>assistant
                                  中国的重点领域产业关键与共性技术攻关工程包括以下几个方面：
                                  
                                  1. 节能环保：主要针对节能减排、资源循环利用、环境污染治理等方面的技术研发。
                                  
                                  2. 智能制造：主要涉及智能制造装备、工业互联网、智能工厂等领域，旨在提高制造业的智能化水平和生产效率。
                                  
                                  3. 人工智能：主要研究机器学习、自然语言处理、计算机视觉等领域的关键技术，以推动人工智能应用的发展。
                                  
                                  4. 生物医药：主要研究药物研发、基因编辑、生物制药等领域，以解决医疗健康领域的问题。
                                  
                                  5. 新材料：主要研究新型材料

中国的重点领域产业关键与共性技术攻关工程包括以下几个方面：

1. 节能环保：主要针对节能减排、资源循环利用、环境污染治理等方面的技术研发。

2. 智能制造：主要涉及智能制造装备、工业互联网、智能工厂等领域，旨在提高制造业的智能化水平和生产效率。

3. 人工智能：主要研究机器学习、自然语言处理、计算机视觉等领域的关键技术，以推动人工智能应用的发展。

4. 生物医药：主要研究药物研发、基因编辑、生物制药等领域，以解决医疗健康领域的问题。

5. 新材料：主要研究新型材料的制备、性能优化等关键技术，以推动新材料在各个领域的应用。

6. 空间科技：主要研究空间探测、卫星通信、航天器设计等领域，以推动空间技术的发展。

7. 物联网：主要研究物联网设备、传感器、网络传输等关键技术，以实现各类物体之间的互联互通。

8. 金融科技：主要研究区块链、云计算、大数据等金融科技核心技术，以推动金融行业的数字化转型。
